In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.python.ops import rnn, rnn_cell
mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)

# Parameters
learning_rate = 0.001
training_iteration_limit = 3
batch_size = 256
display_step = 10

# Network Parameters
total_series_length = 50000
truncated_backprop_length = 15
num_classes = 10
chunk_size = 28
n_chunks = 28
rnn_size = 128

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([rnn_size,num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

x = tf.placeholder('float', [None, n_chunks,chunk_size])
y = tf.placeholder('float')

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
def core_recurrent_neural_net(x):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, chunk_size, n_chunks)

    # Unstack to get a list of 'n_steps' tensors of shape (batch_size, n_input)    
    x = tf.unstack(x, n_chunks, 1)
    
    # Define a lstm cell with tensorflow
    lstm_cell = rnn_cell.BasicLSTMCell(rnn_size, forget_bias=1.0)
    
    # Get lstm cell output
    outputs, states = rnn.rnn(lstm_cell, x, dtype=tf.float32)

     # Linear activation, using rnn inner loop last output
    output = tf.matmul(outputs[-1],weights['out']) + biases['out']

    return output

In [3]:
def train_neural_network(x):
    prediction = core_recurrent_neural_net(x)
    
    # Define loss and optimizer
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(prediction,y) )
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    
   # Launch the graph 
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())

        # Keep training until reach max limit of iteration
        for train_loop in range(training_iteration_limit):
            train_loop_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                train_loop_x, train_loop_y = mnist.train.next_batch(batch_size)
                # Reshape data to get 28 seq of 28 elements
                train_loop_x = train_loop_x.reshape((batch_size,n_chunks,chunk_size))
                # Run optimization op (backprop)
                _, c = sess.run([optimizer, cost], feed_dict={x: train_loop_x, y: train_loop_y})
                train_loop_loss += c
            
            print('train_loop', train_loop, 'completed out of',training_iteration_limit,'loss:',train_loop_loss)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        
        # Evaluate model
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:mnist.test.images.reshape((-1, n_chunks, chunk_size)), y:mnist.test.labels}))
        
        
train_neural_network(x)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
train_loop 0 completed out of 3 loss: 136.217165358
train_loop 1 completed out of 3 loss: 35.722185269
train_loop 2 completed out of 3 loss: 24.1941645443
Accuracy: 0.9717
